In [115]:
!find ../../../workspace -type d -name ".ipynb_checkpoints" -exec rm -rf {} +

In [116]:
!python3 rename.py

In [117]:
!python3 run_example_designs.py --architecture eyeriss_like --problem CONV/assp



Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer3...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer4...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer5...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer2...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer1...
input file: /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer4/parsed-processed-input.yaml
input file: /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/assp_layer3/parsed-processed-input.yaml
input file: /home/wo

In [128]:
!python3 copy_files.py

In [129]:
!python3 run_example_designs.py --architecture eyeriss_like --problem CONV/DeepLabv3



Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer13...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer44...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer52...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer29...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer7...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss_like/outputs/DeepLabv3_layer48...


Running mapper for target eyeriss_like in /home/workspace/final-project/example_designs/example_designs/eyeriss